# My Job Applications 2023

### Imports

In [1]:
import numpy as np
import pandas as pd

### Empty Data Frame

In [12]:
df_columns = {'date_applied': 'datetime64[ns]', 'job_title': 'str', 'company': 'str', 
              'location': 'str', 'date_init_resp': 'datetime64[ns]', 
              'initial_response': 'int8', 'date_interview1': 'datetime64[ns]',
              'interview_comments': 'str', 'more_interviews': 'str', 
              'final_outcome': 'int8'}

jobs_df = pd.DataFrame({c: pd.Series(dtype=t) for c, t in df_columns.items()})
jobs_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   date_applied        0 non-null      datetime64[ns]
 1   job_title           0 non-null      object        
 2   company             0 non-null      object        
 3   location            0 non-null      object        
 4   date_init_resp      0 non-null      datetime64[ns]
 5   initial_response    0 non-null      int8          
 6   date_interview1     0 non-null      datetime64[ns]
 7   interview_comments  0 non-null      object        
 8   more_interviews     0 non-null      object        
 9   final_outcome       0 non-null      int8          
dtypes: datetime64[ns](3), int8(2), object(5)
memory usage: 0.0+ bytes


### Insert Data

In [ ]:
new_jobs_jan = 

jobs_df.append(new_jobs_jan)